In [1]:
import time
import pandas as pd
import numpy as np
import random
import json

In [2]:
class Account:
    def __init__(self, Bank, acc_no, name, age, address, balance=0.0):
        self.acc_no = acc_no
        if age < 18:
            raise ValueError("Enter Age above 18.")
        self.age = age
        self.address = address
        self.name = name
        self.balance = balance
        self.Bank = Bank
        with open(f"statement_{self.acc_no}.txt", "a") as file:
            file.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')}: Account {self.acc_no} was Created Successfully!\n")
        print(f"Account {self.acc_no} was Created Successfully!")
        print(f"Account Number: {acc_no}")
        self.Bank.save_file()

    def deposit(self, Bank, amount):
        self.balance += amount
        with open(f"statement_{self.acc_no}.txt", "a") as file:
            file.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')}: {amount} was credited to Account {self.acc_no} Successfully!\n")
        self.Bank.save_file()
        print(f"Account {self.acc_no} was Updated Successfully!")
        print(f"Updated Balance: {self.balance}")

    def withdraw(self, Bank, amount):
        if amount > self.balance:
            raise ValueError("Insufficient Funds!")
        self.balance -= amount
        with open(f"statement_{self.acc_no}.txt", "a") as file:
            file.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')}: {amount} was debited from Account {self.acc_no} Successfully!\n")
        print(f"Account {self.acc_no} was Updated Successfully!")
        print(f"Remaining Balance: {self.balance}")
        self.Bank.save_file()


    def update_info(self, Bank, name, age, address):
        if age < 18:
          raise ValueError("Enter Age above 18.")
        self.name = name
        self.age = age
        self.address = address
        with open(f"statement_{self.acc_no}.txt", "a") as file:
            file.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')}: Account {self.acc_no} was Updated Successfully!\n")
        print(f"Account {self.acc_no} was Updated Successfully!")
        self.Bank.save_file()

    def show_balance(self):
        print(f"Account Name: {self.name}")
        print(f"Account Balance: {self.balance}")

    def print_statement(self):
        with open(f"statement_{self.acc_no}.txt", "r") as file:
            print(file.read())


In [3]:
class Bank:
  def __init__(self):
    self.accounts = {}
    self.accounts_data = {}
    self.account_sequence = random.randint(100000,200000)
    self._logs = "Banking_Application_logs.json"
    self.load_file()

  def create_account(self, name, age, address, initial_deposit=0.0):
    while self.account_sequence in self.accounts:
      self.account_sequence += 1
    acc_no = self.account_sequence
    self.accounts[acc_no] = Account(self, acc_no, name, age, address, initial_deposit)
    self.account_sequence += 1

  def get_account(self, acc_no):
    try:
      if acc_no in self.accounts:
        return self.accounts[acc_no]
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def update_account(self, acc_no, name, age, address):
    try:
      if acc_no in self.accounts:
        self.accounts[acc_no].update_info(self, name, age, address)
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def delete_account(self, acc_no):
    try:
      if acc_no in self.accounts:
        del self.accounts[acc_no]
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def deposit(self, acc_no, amount):
    try:
      if acc_no in self.accounts:
        self.accounts[acc_no].deposit(self, amount)
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def withdraw(self, acc_no, amount):
    try:
      if acc_no in self.accounts:
        self.accounts[acc_no].withdraw(self, amount)
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def show_balance(self, acc_no):
    try:
      if acc_no in self.accounts:
        self.accounts[acc_no].show_balance()
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def print_statement(self, acc_no):
    try:
      if acc_no in self.accounts:
        self.accounts[acc_no].print_statement()
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def show_all_accounts(self):
    for acc_no, account in self.accounts.items():
      print(f"\nAccount Number: {acc_no}")
      print(f"Name: {account.name}", end = " ")
      print(f"Age: {account.age}", end = " ")
      print(f"Address: {account.address}")
      print(f"Account Balance: {account.balance}")

  def show_account(self, acc_no):
    try:
      if acc_no in self.accounts:
        print(f"Account Number: {acc_no}")
        print(f"Account Name: {self.accounts[acc_no].name}", end = " ")
        print(f"Account Age: {self.accounts[acc_no].age}", end = " ")
        print(f"Account Address: {self.accounts[acc_no].address}")
        print(f"Account Balance: {self.accounts[acc_no].balance}\n")
      else:
        raise KeyError("Account does not exist!")
    except KeyError as e:
      print(f"Error: {e}")

  def save_file(self):
    self.accounts_data = {}
    for acc_no, acc_info in self.accounts.items():
      self.accounts_data[acc_no] = {
        "name": acc_info.name,
        "age": acc_info.age,
        "address": acc_info.address,
        "balance": acc_info.balance
      }
    try:
      with open(self._logs, "w") as file:
        json.dump(self.accounts_data, file, indent = 4)
    except IOError as e:
      print(f"Error: {e}")

  def load_file(self):
    self.accounts = {}
    try:
      with open(self._logs, "r") as file:
        self.accounts_data = json.load(file)
        for acc_no, acc_info in self.accounts_data.items():
          acc_no = int(acc_no)
          age = int(acc_info["age"])
          balance = float(acc_info["balance"])
          self.accounts[acc_no] = Account(self, acc_no, acc_info["name"], age, acc_info["address"], balance)

    except FileNotFoundError:
      print(f"file '{self._logs}' not found")


In [4]:
class manager:
  def __init__(self, bank, passkey):
    self.bank = bank
    self.passkey = passkey
    self.attempt = 3
  def show_all_accounts(self):
      while self.attempt > 0:
        passkey = input("Enter passkey: ")
        if passkey != self.passkey:
          self.attempt -= 1
          print("Incorrect Passkey!")
          time.sleep(0.5)
        else:
          self.bank.show_all_accounts()
          break
      else:
        print("Maximum Attempts Reached!")
  def delete_account(self, acc_no):
    while self.attempt > 0:
      passkey = input("Enter passkey: ")
      if passkey != self.passkey:
        self.attempt -= 1
        print("Incorrect Passkey!")
        time.sleep(0.5)
      else:
        self.bank.delete_account(acc_no)
        break
    else:
      print("Maximum Attempts Reached!")

Introduce caeser cipher to encrypt passkey and store passkey on first line & Manager details in a file <br> define change_passkey

In [5]:
print("1. Create Account")
print("2. Update Account")
print("3. Delete Account")   # Need Manager Access
print("4. Deposit")
print("5. Withdraw")
print("6. Show Balance")
print("7. Print Statement")
print("8. Show my Account")
print("9. Show All Accounts")  # Need Manager Access
print("10. Exit")

bank = Bank()
manager = manager(bank, "1234")

while True:
  try:
    choice = input("\nEnter choice: ")
    if choice == "1":
      name = input("Enter name: ")
      while True:
        try:
          age = int(input("Enter age: "))
          if age < 18:
            raise ValueError("Must be Above 18.")
        except ValueError:
          print("Invalid Age!")
          continue
        else:
          break
      address = input("Enter address: ")
      initial_deposit = float(input("Enter initial deposit: "))
      bank.create_account(name, age, address, initial_deposit)
    elif choice == "2":
      acc_no = int(input("Enter account number: "))
      name = input("Enter name: ")
      while True:
        try:
          age = int(input("Enter age: "))
          if age < 18:
            raise ValueError("Must be Above 18.")
        except ValueError:
          print("Invalid Age!")
          continue
        else:
          break
      address = input("Enter address: ")
      bank.update_account(acc_no, name, age, address)
    elif choice == "3":
      acc_no = int(input("Enter account number: "))
      manager.delete_account(acc_no)
    elif choice == "4":
      acc_no = int(input("Enter account number: "))
      amount = float(input("Enter amount: "))
      bank.deposit(acc_no, amount)
    elif choice == "5":
      acc_no = int(input("Enter account number: "))
      amount = float(input("Enter amount: "))
      bank.withdraw(acc_no, amount)
    elif choice == "6":
      acc_no = int(input("Enter account number: "))
      bank.show_balance(acc_no)
    elif choice == "7":
      acc_no = int(input("Enter account number: "))
      bank.print_statement(acc_no)
    elif choice == "8":
      acc_no = int(input("Enter account number: "))
      bank.show_account(acc_no)
    elif choice == "9":
      manager.show_all_accounts()
    elif choice == "10":
      bank.save_file()
      break
    else:
      print("Invalid Choice!")
  except ValueError:
    print("Invalid Choice!")

1. Create Account
2. Update Account
3. Delete Account
4. Deposit
5. Withdraw
6. Show Balance
7. Print Statement
8. Show my Account
9. Show All Accounts
10. Exit
Account 174547 was Created Successfully!
Account Number: 174547
Account 174548 was Created Successfully!
Account Number: 174548
Account 121893 was Created Successfully!
Account Number: 121893
Account 197240 was Created Successfully!
Account Number: 197240

Enter choice: 9
Enter passkey: 1234

Account Number: 174547
Name: Trijal Khade Age: 18 Address: Pune
Account Balance: 34.0

Account Number: 174548
Name: rija Age: 34 Address: thre
Account Balance: 123.0

Account Number: 121893
Name: Atharva Age: 25 Address: kolhapur
Account Balance: 2900.0

Account Number: 197240
Name: Pranjal Khade Age: 19 Address: Pimpri
Account Balance: 1209.0

Enter choice: 4
Enter account number: 174547
Enter amount: 120
Account 174547 was Updated Successfully!
Updated Balance: 154.0

Enter choice: 5
Enter account number: 197240
Enter amount: 100
Account